## ISIC 2020 IMAGE CLASSIFICATION WITH PYTORCH

This is used for building an image classifier using ISIC 2020 patient centric dataset. This work follows these steps:
1. Get the train csv meta data
2. Perform subsampling to solve the class imabalance problem in the data set.
3. Prepare the data using the sub-sample

### Import needed library

In [108]:
import numpy as np
import pandas as pd
import os
import cv2
import random as rn
import time
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
#load touch libraries
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import torch
# Neural networks can be constructed using the torch.nn package.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms, models

In [109]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(" Device is {}".format(device))

In [110]:

print(os.listdir("../input"))
for (dirpath, dirnames, filenames) in os.walk("../input/"):
    print("Directory path: ", dirpath)
    print("Folder name: ", dirnames)
# for filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [111]:
BASE_PATH = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train'
print(BASE_PATH)
path = os.path.abspath('/kaggle/input/siim-isic-melanoma-classification')
print(path)

## Loading the train meata data and preprocessing it

We load the train.csv file. Then perform the following:

*   Preprocess it to remove null records
*   Plot graph of the dataset 

In [112]:
train_df = pd.read_csv(os.path.join(path,"train.csv"))
train_df.head()

### Preprocessing
Check the dataset for NAs and Null. Then remove nas

In [113]:
print("The number of rows and columne in train.csv: {}".format(train_df.shape))
print(train_df.columns)
print("\nDATA TYPES\n")
print(train_df.dtypes)
print("\n=================================================\n")
print("\nCOLUMN COUNT\n")
print(train_df.count())
print("\nNULL COUNT\n")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

In [114]:
# Covert columns to categorical type. 
# Passed a dictionary to astype() function
train_df2 = train_df.astype({'patient_id':'category',"sex":'category', "diagnosis":'category',
                             'benign_malignant':'category','target':'category',
                             'anatom_site_general_challenge':'category'})

### After preprocessing

In [115]:
print("The number of rows and columne in train.csv: {}".format(train_df2.shape))

### Plotting the dataset
These plots chech for class imbalance between benign and melanoma.
1. Plot the distribution of the age
2. Show image count per patient
3. Plot the number of images according to image_diagnosis, benign or malignant, target 

### Helper functions are defined below

In [116]:
def plot_histogram(data, plot_properties):
  """ Generic function to plot a histogram.
      param: data- is an 1D array of data
      param: plot_properties- a dictionary of properties like:
      {"title":"","xlabel":"","ylabel":""}
  """
  plt.figure(figsize=(10, 6))
  plt.hist(data)
  plt.grid(False)
  plt.title(plot_properties["title"])
  plt.xlabel(plot_properties["xlabel"])
  plt.ylabel(plot_properties["ylabel"])
  plt.show()


def plot_box(data, plot_properties):
  """ Generic function to plot a box plot.
      param: data- is an 1D array of data
      param: plot_properties- a dictionary of properties like:
      {"title":"","xlabel":"","ylabel":""}
  """
  plt.figure(figsize=(10, 6))
  plt.boxplot(data)
  plt.grid(False)
  plt.title(plot_properties["title"])
  plt.xlabel(plot_properties["xlabel"])
  plt.ylabel(plot_properties["ylabel"])
  plt.show()
    
def img_display(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg 

def test_network(net, trainloader):

    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    dataiter = iter(trainloader)
    images, labels = dataiter.next()

    # Create Variables for the inputs and targets
    inputs = Variable(images)
    targets = Variable(images)

    # Clear the gradients from all Variables
    optimizer.zero_grad()

    # Forward pass, then backward pass, then update weights
    output = net.forward(inputs)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()

    return True

In [117]:
plot_histogram(train_df2.age_approx, 
               {"title":"DISTRIBUTION OF APPROXIMAGE AGE IN THE DATASET",
                "xlabel":"APPROX AGE","ylabel":"FREQUENCY"});

In [118]:
patient_group=train_df2.groupby('patient_id').image_name
print("\nNUMBER OF IMAGES PER PATIENT\n")
print(patient_group.describe())
print("\n Statistical description \n")
print(patient_group.count().describe())

In [119]:
plt.figure(figsize=(10, 6))
chart = sns.countplot(train_df2.diagnosis,x="diagnosis",palette='Set2')
chart.set_xticklabels(chart.get_xticklabels(), 
                      rotation=45, horizontalalignment='right')
plt.title("IMAGES BY DIAGNOSIS")
plt.xlabel("ANATOMMIC DIAGNOSIS")
plt.ylabel("NUMBER OF IMAGES")
plt.show()

In [120]:
plt.figure(figsize=(10, 6))
chart = sns.countplot(train_df2.benign_malignant,
    x="benign_malignant",
    palette='Set2'
)
chart.set_xticklabels(chart.get_xticklabels(), 
                      rotation=0, horizontalalignment='right')
plt.title("IMAGES BY GROUND TRUTH")
plt.xlabel("BENIGN_MALIGNANT")
plt.ylabel("NUMBER OF IMAGES")
plt.show()

#### Malignant data
We select all records from the dataset that are malignant.
We get the number of records
We group by patients and count the number of patients.
We count the number of images per patient

In [121]:
# selecting rows based on condition
malignant_df = train_df2[train_df2['target'] == 1]
print("The number of malignant records are:",malignant_df.shape)
malignant_patient_group=malignant_df.groupby('patient_id').target
patient_count=malignant_patient_group.count()
malignant_patient_count=patient_count.where(patient_count>0).dropna()
malignant_patient_data=np.column_stack((malignant_patient_count.index,malignant_patient_count))
print("\nStatistical description\n")
print(malignant_patient_count.describe())
plot_box(malignant_patient_data[:,1], 
{"title":"DISTRIBUTION OF NUMBER OF MALIGNANT IMAGES PER PATIENT","xlabel":"NUMBER OF IMAGES PER PATIENT","ylabel":"FREQUENCY"});

In [122]:
plot_histogram(malignant_patient_data[:,1], 
{"title":"DISTRIBUTION OF NUMBER OF MALIGNANT IMAGES PER PATIENT","xlabel":"NUMBER OF IMAGES PER PATIENT","ylabel":"FREQUENCY"});

#### Benign dataset
We select all records from the dataset that are malignant.
We get the number of records
We group by patients and count the number of patients.
We count the number of images per patient

In [123]:
# selecting rows based on condition
benign_df = train_df2[train_df2['target'] == 0]
print("The number of benign records are:",benign_df.shape)
benign_patient_group=benign_df.groupby('patient_id').target
b_patient_count=benign_patient_group.count()
benign_patient_count=b_patient_count.where(b_patient_count>0).dropna()
benign_patient_data=np.column_stack((benign_patient_count.index,benign_patient_count))
print("\nStatistical Description\n")
print(benign_patient_count.describe())
plot_box(benign_patient_data[:,1], 
{"title":"DISTRIBUTION OF NUMBER OF BENIGN IMAGES PER PATIENT","xlabel":"NUMBER OF IMAGES PER PATIENT","ylabel":"FREQUENCY"});

In [124]:
plot_histogram(benign_patient_data[:,1], 
{"title":"DISTRIBUTION OF NUMBER OF BENIGN IMAGES PER PATIENT","xlabel":"NUMBER OF IMAGES PER PATIENT","ylabel":"FREQUENCY"});

### Subsampling
Based on the dataset, we start by selecting a single image per patient that is selecting images by patient Id. This is used to select images from the original image set for training and classification.

Subsampling is done by checking getting the patient images that are malignant or benign. If the patient id has malignant samples, we sample 1. Else if the patient id has benign sample, we sample 1. This is done to make the dataset have enough representation of malignant as well as benign data while representing every patient.

In [125]:
# set seed for reproducability
seed_value= 12321 
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
rn.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

In [126]:
# Get patient id as category
# Prepare two datasets:
# for each patient_id
    # select get records
      # if malignant
        # sample 1
      # elseif benign
        # sample 1

unique_patient_id = train_df2.patient_id.cat.categories
malignant_df = train_df2[train_df2['target'] == 1]

benign_df = train_df2[train_df2['target'] == 0]
sample_indices = []
#samples per patient
for  patient_id in unique_patient_id:
  patient_sample = malignant_df[malignant_df['patient_id']==patient_id]
  if patient_sample.size:
     sample_indices.append(patient_sample.sample(n=1, random_state=seed_value).index[0])
  else:
     patient_sample = benign_df[benign_df['patient_id']==patient_id]
     if patient_sample.size: 
       sample_indices.append(patient_sample.sample(n=1, random_state=seed_value).index[0])

sample_df = train_df2.loc[sample_indices]
print("Sample length",len(sample_indices))
sample_df.tail()

### Exploring and Visualizing the sub-sample
The following is done in this section:
1. Count the samples by benign or malignant
2. Plot the count

In [127]:
sample_df.groupby('benign_malignant').count()

In [128]:
plt.figure(figsize=(10, 6))
chart = sns.countplot(sample_df.benign_malignant,x="benign_malignant",palette='Set2')
chart.set_xticklabels(chart.get_xticklabels(), 
                      rotation=0, horizontalalignment='right')
plt.title("IMAGES BY GROUND TRUTH")
plt.xlabel("BENIGN_MALIGNANT")
plt.ylabel("NUMBER OF IMAGES")
plt.show()

## Prepare the data for classification
In this section, we load the images by converting them to pytouch

In [129]:
def set_image_file_name(row):
    """We create the image file name"""
    file = row["image_name"]+".jpg"
    return file if os.path.exists(os.path.join(BASE_PATH, file)) else np.nan

sample_df['images'] =  sample_df[['image_name','benign_malignant']].apply(lambda row: set_image_file_name(row), axis=1)
sample_df.head()

### Splitting the dataset


In [130]:
batch_size = 100
validation_split = .3
shuffle_dataset = True
random_seed= 42

In [131]:
# sample_df.benign_malignant.unique()
dataset_size = len(sample_df)
indices = sample_df.index.tolist()
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

### Test Sub-sample

In [167]:
# Seperate indices that are not in the sample dataframe
train_df_indices = np.array(train_df.index)
ind=np.where(np.invert(np.isin(train_df_indices,indices)))
test_indices = train_df_indices[ind]
test_sample_indices=np.random.choice(test_indices,100)
test_sample_df = train_df2.loc[test_sample_indices]

In [168]:
test_sample_df['images'] =  test_sample_df[['image_name','benign_malignant']].apply(lambda row: set_image_file_name(row), axis=1)
test_sample_df.head()

In [169]:
test_sampler = SubsetRandomSampler(test_sample_indices)

### Transforms
Transforms are common image transformations. They can be chained together using Compose.

**Normalization**
Normalize a tensor image with mean and standard deviation. Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input torch.*Tensor i.e. input[channel] = (input[channel] - mean[channel]) / std[channel] 

**Convert a PIL Image or numpy.ndarray to tensor**
Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or if the numpy.ndarray has dtype = np.uint8

In [135]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### Create custom dataset class

> A custom dataset which contain following functions is defined to be used by data loader later on.
1. init() function where the initial logic happens.
2. getitem() function returns the data and labels.

In [136]:
class Lesion_Dataset(Dataset):
    def __init__(self, img_data,img_path,transform=None):
        self.img_path = img_path
        self.transform = transform
        self.img_data = img_data
        
    def __len__(self):
        return len(self.img_data)
    
    def __getitem__(self, index):
        img_name = os.path.join(self.img_path,self.img_data.loc[index, 'images'])
        image = Image.open(img_name)
        #image = image.convert('RGB')
        image = image.resize((300,300))
        label = torch.tensor(self.img_data.loc[index, 'target'])
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [137]:
# Create the dataset
dataset = Lesion_Dataset(sample_df,BASE_PATH,transform)

In [170]:
test_dataset = Lesion_Dataset(test_sample_df,BASE_PATH,transform)

In [138]:
# create our loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                                sampler=valid_sampler)

In [171]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                                sampler=test_sampler)
print("Training and validation loader completed ")

In [ ]:
# let's see the number of positive and negative image
# a,b=np.unique(df["target"],return_counts=True)

# from sklearn.model_selection import train_test_split
# train,valid=train_test_split(df,test_size=0.2)

# #now split train to get some image for testing
# train,test=train_test_split(train,test_size=.01)

### Visualizing images in the dataset


In [139]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

lesion_types={0:'benign', 1:'malignant'} 
fig, axis = plt.subplots(3, 5, figsize=(15, 10))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label = images[i], labels[i]
        ax.imshow(img_display(image)) # add image
        ax.set(title = f"{lesion_types[label.item()]}")

## Performing classification

Here we perform classification using:
1. Transfer learning: with a model such as DenseNet. Let's print out the model architecture so we can see what's going on.

In [140]:
model = models.densenet121(pretrained=True)
# model
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

### Training

In [141]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(" Device is {}".format(device))

CUDA GPU is used instead of cuda for training

In [142]:
# for device in ['cpu', 'cuda']:

#     criterion = nn.NLLLoss()
#     # Only train the classifier parameters, feature parameters are frozen
#     optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

#     model.to(device)

#     for ii, (inputs, labels) in enumerate(train_loader):

#         # Move input and label tensors to the GPU
#         inputs, labels = inputs.to(device), labels.to(device)

#         start = time.time()

#         outputs = model.forward(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         if ii==3:
#             break
        
#     print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

In [143]:
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [146]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in validation_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(validation_loader):.3f}.. "
                  f"Test accuracy: {accuracy/len(validation_loader):.3f}")
            running_loss = 0
            model.train()

## Evaluating the model performance
Here, the following is done.
1. Sample a test data from the original dataset.
2. Test the model for accuracy

In [172]:
dataiter_test = iter(test_loader)
test_images, test_labels = dataiter_test.next()
test_inputs = test_images.to(device)

In [173]:
outputs = model(test_inputs)

In [174]:
classes  = ("benign", "malignant")

Using the test dataset

In [175]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', f'{classes[predicted[0]]:5s}')
print('Actual label: ', f'{classes[test_labels[0]]:5s}')

In [176]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        test_images, test_labels = data
        test_inputs, input_labels = test_images.to(device), test_labels.to(device)
        outputs = model(test_inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(input_labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

In [178]:
total_pred

In [179]:
correct_pred

In [180]:
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    print(classname,correct_count)
    accuracy = 0 if correct_count == 0 else 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Save the model